In [0]:
import re
import sys
import time


```
There are some global variable whiach are used in entire code
* LitCalMatrix stores the index of clauses, for all literals there is a array 
store index of clause
* literalCount stores the Number of literals
* unknown is a Global value... for unassign literals
* litValues stores the current assign value of literal, 0 = False, 1 = True, -1 = Uknown or unassign
* formula is a cnf or list of clauses and each clause is list of literals
* level is number of manually assign literals
* levelAllLits maintain level wize literals assignment
* adjMatrix is the adjecent list of graph used in conflict analysis
* DecisionCount, ImplicationCount are global Int

```



In [0]:
LitCalMatrix = {}
literalCount = 0
unknown = -1
litValues = []
formula = []
level = 0
levelAllLits = {}
adjMatrix = []
DecisionCount = 0
ImplicationCount = 0

Given a index of formula, output the value of clause true, false or unknown
  - if any literal is satisfiable then clause is true
  - if no one literal is satisfiable then any literal is unknown then unknown 
  - if all literal are false then clause is false



In [0]:
def ClauseValue(index):
    answer = 0
    for lit in formula[index]:
        if (lit > 0 and litValues[abs(lit)]==1) or (lit < 0 and litValues[abs(lit)]==0):
            return 1
        elif litValues[abs(lit)]==unknown:
            answer = unknown
    return answer

Given a unknown valued clause, find out the number of unknown literals.
It output the number of unknown literal and return atleast one literal.

In [0]:
def NumOfUnknownLit(clause):
    count = 0
    resLit = 0
    for lit in clause:
        currValue = litValues[abs(lit)]
        if currValue == unknown:
            count = count + 1
            resLit = lit
    return count,resLit

Whenever any new Literal is assign through unit propogation, we have to add new node in graph or update adjacency list of graph.
* assign value to literal
* update value, level, clause Index in graph

In [0]:
def UpdateDataStr(literal,index):
    pstLit = abs(literal)
    if literal>0:
        litValues[pstLit] = 1
    else:
        litValues[pstLit] = 0

    adjMatrix[pstLit]["value"] = litValues[pstLit]
    adjMatrix[pstLit]["level"] = level
    adjMatrix[pstLit]["clause"] = index

In BCP function... basically we filter the clause in which all literal are false except one unknown, that literal shouls be true.

* Given all possible set of clause Index whcih are unit clause
* Iterate all given clause and check number of unknown literal in unknown valued clause
* If number of unknown literal is one then stre them with clause index
** if we not get any literal the stop it
**  else may be we get more unit literal through these literals
* for each unit literal we add node in graph and its value
* we get more unit literal only me that clause which have negative of them

In [0]:
def BCP(possibleCalIdx):
    global ImplicationCount
    unitLitsPair = set()
    unitLits = set()
    for i in possibleCalIdx:
        calVal = ClauseValue(i)
        if(calVal==unknown):
            cnt,lit = NumOfUnknownLit(formula[i])
            if cnt==1:
                ImplicationCount = ImplicationCount +1
                unitLitsPair.add((lit,i))
                unitLits.add(lit)
        elif calVal==0:
            return i

    if len(unitLitsPair)==0:
        return -1
    else:
        for lit,idx in unitLitsPair:
            if level==0:
                litValues[abs(lit)] = 1 if lit>0 else 0
                adjMatrix[abs(lit)]["value"] = litValues[abs(lit)]
                adjMatrix[abs(lit)]["level"] = level
                adjMatrix[abs(lit)]["clause"] = idx
            else:
                levelAllLits[level].append(lit)
                UpdateDataStr(lit,idx)
        
        newPossibleCalIdx =set()

        for lit in unitLits:
            if -1*lit in LitCalMatrix.keys():
                for idx in LitCalMatrix[-1*lit]:
                    newPossibleCalIdx.add(idx)
        return BCP(newPossibleCalIdx)

Decide function decide which next unknown literal be true
We iterate at all uknown literals and return **max occuring literal in formula**

In [0]:
def decide():
    global DecisionCount
    DecisionCount = DecisionCount+1
    ansLit = 0
    count =0
    for lit in LitCalMatrix.keys():
        if litValues[abs(lit)]==unknown:
            if len(LitCalMatrix[lit]) > count:
                count =len(LitCalMatrix[lit]) 
                ansLit = lit
    return ansLit

Branching function create a new branch, take one new decision
* increment level by one
* choose literal by decide function
* update data structue like add new node in graph and litValues

In [0]:
def Branching():
    global level
    level = level+1
    lit = decide()
    litValues[abs(lit)]= 1 if lit > 0 else 0
    levelAllLits[level] = []
    levelAllLits[level].append(lit)
    adjMatrix[abs(lit)]["level"] = level
    adjMatrix[abs(lit)]["value"] = litValues[lit]
    adjMatrix[abs(lit)]["clause"] = None
    

During Conflict we have to jump to new level which is less than current level
* All literals level wise stored in levelAllLits dict
* for each level between new level and current level
** set literal value as unknown
** remove node form graph
** clear levelAllLits dict


In [0]:
def UpdateDataStrWithNewLevel(newLevel):

    for lvl in range(newLevel+1,level+1):
        for lit in levelAllLits[lvl]:
            litValues[abs(lit)]=unknown
            adjMatrix[abs(lit)]={}

    for lvl in range(newLevel+1,level+1):
        levelAllLits[lvl] = []        


To resolve conflict in givel index at current level
* iterate form leaf node to root node of current level
** for each node differenciate current level literals and perv level literals
** if current level lit is one then break
** else for each lit in clause of leaf node
** all not done literals used in next iteration

At end we get a new clause by adding all pervious Lits an current lits
* new level is the maximum of all pervious lits level
* return the new level and new clause

In [0]:
def ConflictAnalyze(clauseIdx):

    clause = formula[clauseIdx]
    poolLiterals = clause
    currentLevelLits=set()
    prevLevelLits=set()
    allAncestorLits=set()

    while True:
        for lit in poolLiterals:
            if adjMatrix[abs(lit)]['level']==level:
                currentLevelLits.add(lit)
            else:
                prevLevelLits.add(lit)

        if len(currentLevelLits)==1:
            break

        for lit in reversed(levelAllLits[level]):
            if lit in currentLevelLits:
                leafLit = lit
                currentLevelLits.remove(lit)
                break

            if -1*lit in currentLevelLits:
                leafLit = -1*lit
                currentLevelLits.remove(-1*lit)
                break

        allAncestorLits.add(abs(leafLit))
        calIdx = adjMatrix[abs(leafLit)]["clause"]
        pervCal = formula[calIdx]

        poolLiterals=[]
        if pervCal is not None:
            for l in pervCal :
                if abs(l) not in allAncestorLits:
                    poolLiterals.append(l)

    newLearnClause = []
    for lit in currentLevelLits.union(prevLevelLits):
        newLearnClause.append(lit)

    if prevLevelLits:
        newLevel = max([adjMatrix[abs(lit)]["level"] for lit in prevLevelLits])
    else:
        newLevel = level - 1

    return newLearnClause, newLevel


This is the main function

* First we take the input line by line from file given in argument
> * if line is starting with c then ignore comments
> * if line start with p then 3 and 4 integer are clause count and literal count
> * rest all line are clauses containing 0 at end

* set unknown value and NULL node for each literal

here is a **optimization**
 If there only lit in occur in formula and negetion of lit in not ocuring then set lit as **TRUE**

 * firstly we call BCP for all clauses and then we have 2 possibilities there is any conflict or no more unit literals

 > * if Conflict occur 
 >>* if current level is 0 then SAT is not possible then return UNSAT
 >>*  resolve conflit and find new level and new clause
 >>*  Update Data Str with new clause and backtrack to new level
>* if all clause are ture then we get SAT solution
>* If we not get any conflict and not all clause true 
>>* choose an unknown literal by decide function and assign that literal true
>>* update data str like add new node in graph and increase level

Finally print all data
* Time spent
* New clause learn
* SAT ot UNSAT
* Literal Values



In [11]:
if __name__ == "__main__":
    currentTime = time.time()
    Answer = 0
    # if len(sys.argv) != 2:
    #     sys.exit("Invalid Command... please mentioned FileName and InputFilePath")
    InputFilePath = "input2.cnf"
    inputFile = open(InputFilePath,'r')
    inputFileData = inputFile.readlines()

    for lines in inputFileData:
        currentLine = lines.strip()
        if(currentLine[0]=='c'):
            continue
        elif (currentLine[0]=='p'):
            currentLineSplit = currentLine.split()
            literalCount = int(currentLineSplit[2])
            clauseCount = int(currentLineSplit[3])
        else:
            currentLineSplit = currentLine.split()
            clause=set()
            finalClause = []
            for lit in currentLineSplit:
                intLit = int(lit)
                if intLit != 0:
                    clause.add(intLit)
            
            for lit in clause:
                if not(-1*lit in clause):
                    if lit in LitCalMatrix.keys():
                        LitCalMatrix[lit].append(len(formula))
                    else:
                        LitCalMatrix[lit] = [len(formula)]
                    finalClause.append(lit)
                    
            formula.append(clause)

    litValues.append(unknown)
    adjMatrix.append({})
    for lit in range(1,literalCount+1):
        litValues.append(unknown)
        adjMatrix.append({})
    
    for key in LitCalMatrix:
        if not(-1*key in LitCalMatrix.keys()):
            pstLit = abs(key)
            litValues[pstLit] = 1 if key > 0 else 0
            adjMatrix[pstLit]["value"] = litValues[pstLit]
            adjMatrix[pstLit]["level"] = level
            adjMatrix[pstLit]["clause"] = None

    NewClauseLearnCount=0
    while True:
        allClauseSet = list(range(0,len(formula)))
        resClauseIdx = BCP(allClauseSet)
        if resClauseIdx >-1:
            if level==0:
                Answer=0
                break

            newClause,newLevel = ConflictAnalyze(resClauseIdx)
            if(newLevel<0):
                Answer=0
                break
            NewClauseLearnCount=NewClauseLearnCount+1

            clauseCount = clauseCount+1
            for lit in newClause:
                LitCalMatrix[lit].append(len(formula))
            formula.append(newClause)
            UpdateDataStrWithNewLevel(newLevel)
            level=newLevel
        elif min([ClauseValue(i) for i in range(0,clauseCount) ])==1:
            Answer=1
            break
        else:
            Branching()
        
        
    print("Time : ", time.time()-currentTime, " second")
    print("Number of Learned Clause: ",NewClauseLearnCount )
    print("Decision : ", DecisionCount)
    print("Implication : ", ImplicationCount)
    if Answer==1:
        for i in range(0,literalCount+1):
            if litValues[i]==-1:
                litValues[i]=1

        print("SAT")
        print(str(litValues[1:]))
        # outFile = open("output.txt", "+w")
        # outFile.write(str(litValues[1:]))
    else:
        print("UNSAT")

Time :  0.00042891502380371094  second
Number of Learned Clause:  0
Decision :  7
Implication :  2
SAT
[0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1]
